In [ ]:
!pip install annoy

In [ ]:
import os
import json
from tqdm.notebook import tqdm
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
from annoy import AnnoyIndex

In [ ]:
# Задаем параметры Annoy
embedding_dim = 512  # Размерность эмбеддинга
index = AnnoyIndex(embedding_dim, 'angular')  # Используем косинусное расстояние для Annoy

# Загружаем модель CLIP
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# Указываем путь к корневой директории и файлу для сохранения метаданных
root_directory = "/kaggle/input/rkn-dataset/train_dataset_train_data_rkn/train_data_rkn/dataset"
metadata_file = "annoy_metadata.json"

In [ ]:
# Создаем словарь для хранения метаданных
metadata = {}
index_counter = 0

# Проход по всем файлам в поддиректориях
for dirpath, _, filenames in tqdm_notebook(os.walk(root_directory)):
    for filename in filenames:
        # Проверка расширения файла (если файл является изображением)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            file_path = os.path.join(dirpath, filename)
            dir_name = os.path.basename(dirpath)  # Название поддиректории
            try:
                # Открываем изображение
                image = Image.open(file_path).convert("RGB")

                # Преобразуем изображение в вектор с помощью CLIP
                inputs = processor(images=image, return_tensors="pt")
                with torch.no_grad():
                    image_embedding = model.get_image_features(**inputs)

                # Конвертируем тензор в numpy для совместимости с Annoy
                image_embedding = image_embedding.cpu().numpy().flatten()

                # Добавляем вектор в индекс Annoy
                index.add_item(index_counter, image_embedding)

                # Сохраняем метаданные
                metadata[index_counter] = {
                    "filename": filename,
                    "directory": dir_name
                }
                index_counter += 1

            except Exception as e:
                print(f"Ошибка обработки изображения {file_path}: {e}")

# Строим индекс Annoy с 10 деревьями (параметр можно регулировать для повышения точности)
index.build(10)

# Сохраняем Annoy-индекс и метаданные
index.save("image_embeddings.ann")
with open(metadata_file, 'w') as f:
    json.dump(metadata, f)